In [1]:
import re
from collections import defaultdict


def initialize_board(board, robots):
    for robot in robots:
        p0, p1, _, _ = robot
        board[p1] = board[p1][:p0] + '1' + board[p1][p0+1:]
    return board


def update_board(board, prev, c_prev, cur, c_cur):
    c_prev = '.' if c_prev == '0' else c_prev
    board[prev[1]] = board[prev[1]][:prev[0]] + c_prev + board[prev[1]][prev[0]+1:]
    board[cur[1]] = board[cur[1]][:cur[0]] + c_cur + board[cur[1]][cur[0]+1:]
    return board


def is_tree_detected(positions, tree_size):
    l_coordinates = sorted([k for k, v in positions.items() if v >= 1], key=lambda x: (x[1], x[0]))
    c_consecutive = 1
    prev_p0, prev_p1 = l_coordinates[0]
    for p0, p1 in l_coordinates[1:]:
        if p0 == prev_p0 + 1 and p1 == prev_p1:
            c_consecutive += 1
            if c_consecutive == tree_size:
                return True
        else:
            c_consecutive = 1
        prev_p0, prev_p1 = p0, p1
    return False


def solve_part_1(path, shape, seconds):
    with open(path) as f:
        robot_instructions = f.read().splitlines()
    q1 = q2 = q3 = q4 = 0
    for robot in robot_instructions:
        p0, p1, v0, v1 = re.findall(r'p=(\d+),(\d+) v=(\-?\d+),(\-?\d+)', robot)[0]
        p0, p1, v0, v1 = int(p0), int(p1), int(v0), int(v1)
        position = (p0 + v0*seconds) % shape[0], (p1 + v1*seconds) % shape[1]
        if position[0] <= shape[0]//2 - 1 and position[1] <= shape[1]//2 - 1:
            q1 += 1
        elif position[0] >= shape[0]//2 + 1 and position[1] <= shape[1]//2 - 1:
            q2 += 1
        elif position[0] <= shape[0]//2 - 1 and position[1] >= shape[1]//2 + 1:
            q3 += 1
        elif position[0] >= shape[0]//2 + 1 and position[1] >= shape[1]//2 + 1:
            q4 += 1
    return q1*q2*q3*q4


def solve_part_2(path, shape, seconds, size):
    with open(path) as f:
        robot_instructions = f.read().splitlines()
    robots = []
    for robot in robot_instructions:
        p0, p1, v0, v1 = re.findall(r'p=(\d+),(\d+) v=(\-?\d+),(\-?\d+)', robot)[0]
        robots.append([int(p0), int(p1), int(v0), int(v1)])
    positions = defaultdict(int)
    for robot in robots:
        p0, p1, v0, v1 = robot
        positions[(p0, p1)] += 1
    board_row = '.' * shape[0]
    board = []
    for row in range(shape[1]):
        board.append(board_row)
    initialize_board(board, robots)
    for second in range(1, seconds + 1):
        for robot in robots:
            p0, p1, v0, v1 = robot
            prev = p0, p1
            cur = (p0 + v0) % shape[0], (p1 + v1) % shape[1]
            robot[0] = cur[0]
            robot[1] = cur[1]
            positions[prev] = max(0, positions[prev] - 1)
            positions[cur] += 1
            c_prev = str(positions[prev])
            c_cur = str(positions[cur])
            update_board(board, prev, c_prev, cur, c_cur)
        if is_tree_detected(positions, size):
            return second, board
    return None


print(solve_part_1('example.txt', (11, 7), 100))
print(solve_part_1('input.txt', (101, 103), 100))
print(solve_part_2('input.txt', (101, 103), seconds=10_000, size=10))

12
220971520
(6355, ['1...................................................................................................1', '........................1...................................1........................................', '....................1..........1.............................................................1.....1.', '.....................................................................................................', '....................................................1............1.........1.................1.......', '.....................1.............1.......................1.........................................', '.........................................................1......................1....................', '.....................1...............................................................................', '......................1.......................................................1......................', '..............1..........1.......